# Cluster de  Keywords V2.
### K-means Implementation

In [1]:
#Lectura de Librerías
import pandas as pd
import time
pd.set_option('display.max_columns', None)

In [2]:
#Leyendo los abstracts
df = pd.read_csv(r'Data\scopus_bd.csv',sep=',')

C:\Users\YORLEY\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (20,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head(1)

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,Page count,Cited by,DOI,Link,Affiliations,Authors with affiliations,Abstract,Author Keywords,Index Keywords,ISSN,ISBN,CODEN,PubMed ID,Document Type,Publication Stage,Access Type,Source,EID
0,Vanzolini P.E.,6602373241;,"On Gymnodactylus amarali Barbour, 1925, with t...",2005,Anais da Academia Brasileira de Ciencias,77,4,NaN,595,611,NaN,7.0,NaN,https://www.scopus.com/inward/record.uri?eid=2...,"Museu de Zoologia, Universidade de São Paulo, ...","Vanzolini, P.E., Museu de Zoologia, Universida...","Gymnodactylus amarali Barbour, 1925, was previ...",Lizards; Speciation; Systematics,animal; article; Brazil; classification; compa...,13765.0,NaN,NaN,1.63414e+07,Article,Final,NaN,Scopus,2-s2.0-31044450573


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52002 entries, 0 to 52001
Data columns (total 28 columns):
Authors                      52002 non-null object
Author(s) ID                 52002 non-null object
Title                        52002 non-null object
Year                         52002 non-null int64
Source title                 52002 non-null object
Volume                       50956 non-null object
Issue                        50618 non-null object
Art. No.                     3184 non-null object
Page start                   47527 non-null object
Page end                     46564 non-null object
Page count                   331 non-null float64
Cited by                     32595 non-null float64
DOI                          26533 non-null object
Link                         52002 non-null object
Affiliations                 50159 non-null object
Authors with affiliations    51612 non-null object
Abstract                     52002 non-null object
Author Keywords           

## Remover Stopwords 

In [5]:
import nltk
from nltk import *
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\YORLEY\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
stop = stopwords.words('english')

In [7]:
dfk = df[['Title','Author Keywords']].drop_duplicates()
dfk["keyw"] = dfk["Author Keywords"].str.lower().str.replace(';','').str.split()

In [8]:
dfk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51511 entries, 0 to 52001
Data columns (total 3 columns):
Title              51511 non-null object
Author Keywords    41708 non-null object
keyw               41708 non-null object
dtypes: object(3)
memory usage: 1.6+ MB


In [9]:
#Borrado de ddtaframe para liberar memoria
del(df)

In [10]:
dfk=dfk[dfk['keyw'].isna()==False]
dfk.reset_index(inplace=True)

In [11]:
dfk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41708 entries, 0 to 41707
Data columns (total 4 columns):
index              41708 non-null int64
Title              41708 non-null object
Author Keywords    41708 non-null object
keyw               41708 non-null object
dtypes: int64(1), object(3)
memory usage: 1.3+ MB


In [12]:
start = time.time()
dfk['clean_keyw'] = dfk['keyw'] \
.apply(lambda x: [item for item in x if item not in stop])
end = time.time()
print((end-start)/60)

0.030546959241231283


In [13]:
dfk['clean_keyw'] = dfk['keyw'] \
.apply(lambda x: ' '.join([word for word in x]))

In [14]:
dfk.isna().any()

index              False
Title              False
Author Keywords    False
keyw               False
clean_keyw         False
dtype: bool

The common way of doing this is to transform the documents into tf-idf vectors, 
then compute the cosine similarity between them. Any textbook on information retrieval (IR) covers this. See esp. Introduction to Information Retrieval, which is free and available online.

Tf-idf (and similar text transformations) are implemented in the Python packages
Gensim and scikit-learn. In the latter package, computing cosine similarities is as easy as


In [15]:

from sklearn.feature_extraction.text import TfidfVectorizer


In [16]:
start=time.time()
keys = dfk['clean_keyw']
#stopwords_ = [word.decode('utf-8') for word in stopwords.words('english')]
tfidf = TfidfVectorizer().fit_transform(keys)
# no need to normalize, since Vectorizer will return normalized tf-idf
end = time.time()
print((end-start)/60)

0.018251105149586996


### Otras posibles Opciones
https://stackoverflow.com/questions/51591510/text-similarity-approaches-do-not-reflect-real-similarity-between-texts
https://stackoverflow.com/questions/8897593/how-to-compute-the-similarity-between-two-text-documents
https://stackoverflow.com/questions/101569/algorithm-to-detect-similar-documents-in-python-script

#Apuntes
#https://stackoverflow.com/questions/25443802/unicode-warning-when-using-nltk-stopwords-with-tfidfvectorizer-of-scikit-learn

In [17]:
from sklearn.cluster import KMeans

# La siguiente son las actividades que más recursos consumen

In [18]:
temp_dict={}

In [19]:
start = time.time()
columnas=[]
for i in range(3,20,1):
    nombre="C_" +str(i)
    columnas.append(nombre)
    kmeans = KMeans(n_clusters=i).fit(tfidf)
    temp_dict[nombre]=kmeans.labels_
end = time.time()
print((end-start)/60)

65.06922274430593


In [20]:
df_cluster=pd.DataFrame(temp_dict)

In [21]:
df_cluster

,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,C_11,C_12,C_13,C_14,C_15,C_16,C_17,C_18,C_19
0,2,0,0,0,0,4,8,0,8,0,0,9,2,0,0,7,8
1,1,1,4,5,1,2,3,4,3,5,3,10,8,13,2,14,6
2,2,0,0,0,0,4,8,0,8,0,0,9,2,5,4,7,14
3,2,0,0,0,0,4,8,0,8,0,0,9,2,0,16,7,8
4,2,0,0,0,0,4,8,9,8,0,0,9,12,6,0,7,8
5,2,0,0,0,0,4,8,0,8,0,0,9,2,0,0,7,8
6,2,0,0,0,0,4,8,0,8,0,0,9,2,0,0,7,8
7,2,0,0,0,0,4,8,0,8,0,0,9,2,0,0,7,8
8,2,0,0,0,0,4,8,0,8,0,0,9,2,0,0,7,13
9,2,0,0,0,0,4,8,0,8,0,0,9,2,0,0,7,8


In [22]:
dfk2 = pd.concat([dfk,df_cluster],axis=1)

In [23]:
dfk2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41708 entries, 0 to 41707
Data columns (total 22 columns):
index              41708 non-null int64
Title              41708 non-null object
Author Keywords    41708 non-null object
keyw               41708 non-null object
clean_keyw         41708 non-null object
C_3                41708 non-null int32
C_4                41708 non-null int32
C_5                41708 non-null int32
C_6                41708 non-null int32
C_7                41708 non-null int32
C_8                41708 non-null int32
C_9                41708 non-null int32
C_10               41708 non-null int32
C_11               41708 non-null int32
C_12               41708 non-null int32
C_13               41708 non-null int32
C_14               41708 non-null int32
C_15               41708 non-null int32
C_16               41708 non-null int32
C_17               41708 non-null int32
C_18               41708 non-null int32
C_19               41708 non-null int32
dtypes:

In [24]:
dfk2.to_csv(r'Data/keywords_clusters.csv', sep=',')